# Quasistatic Planar Pushing A B Matrix

## Imports

In [1]:
import time
import os
import numpy as np

from pydrake.all import Box, Sphere

from qsim.parser import (
    QuasistaticParser,
    QuasistaticSystemBackend,
    GradientMode,
)

from qsim.simulator import ForwardDynamicsMode, InternalVisualizationType
from qsim.model_paths import models_dir

/home/shaoyuan/Documents/Software/quasistatic_simulator/qsim/simulator.py:38: DrakeDeprecationWarning: Please import from the pydrake.solvers module directly, instead of the deprecated pydrake.solvers.mathematicalprogram submodule. The deprecated code will be removed from Drake on or after 2023-05-01.
  from pydrake.solvers import mathematicalprogram as mp


## Initialize Quasistatic Simulator

In [2]:
q_model_path = os.path.join(models_dir, "q_sys", "box_pushing.yml")
q_parser = QuasistaticParser(q_model_path)
q_sim = q_parser.make_simulator_cpp()
q_sim_py = q_parser.make_simulator_py(InternalVisualizationType.Cpp)
q_sim_batch = q_parser.make_batch_simulator()

INFO:drake:Meshcat listening for connections at http://localhost:7002


# Unwrapped version (Plain quasistatic sim)

In [3]:
sim_params = q_sim.get_sim_params_copy()
sim_params.h =0.1
sim_params.use_free_solvers = False
sim_params.gradient_mode = GradientMode.kAB
sim_params.forward_mode = ForwardDynamicsMode.kLogIcecream
sim_params.unactuated_mass_scale = 10
sim_params.log_barrier_weight = 100

q_u = np.array([0, 0, 0])
q_a = np.array([-1, 0])
q = np.concatenate((q_u, q_a)) # [box_x, box_y, box_theta, sphere_x, sphere_y]
u = q_a
q_sim_py.update_mbp_positions_from_vector(q)
q_sim_py.draw_current_configuration()

q_next = q_sim.calc_dynamics(q, u, sim_params)
A = q_sim.get_Dq_nextDq()
B = q_sim.get_Dq_nextDqa_cmd()
print(A)

[[ 9.90553945e-01  7.21210078e-14 -3.89032534e-14 -1.73091159e-02
  -1.61508843e-14]
 [ 8.05134199e-14  1.00750154e+00  7.68977989e-03 -2.45432475e-14
  -2.75574873e-02]
 [-1.87220248e-12  2.18569667e-01  7.81430334e-01 -6.54942341e-13
  -4.37494156e-02]
 [ 9.44605539e-04 -7.21208574e-15  3.89031005e-15  1.73091159e-03
   1.61508584e-15]
 [-8.05134039e-15 -7.50153696e-04 -7.68977835e-04  2.45434049e-15
   2.75574818e-03]]


In [4]:

N = 1000
w_std = 0.1
dim_q = 2

w = np.random.normal(0, w_std, (N,dim_q))

q_u_batch = np.tile(q_u, (N,1))
q_a_batch = np.tile(q_a, (N,1)) + w
q_batch = np.hstack((q_u_batch, q_a_batch))
u_batch = np.tile(u, (N,1))

# print("q_u_batch\n", q_u_batch)
# print("q_a_batch\n", q_a_batch)
# print("q_batch\n", q_batch)
# Option 1
# q_next_batch, A_batch, B_batch, is_valid_batch = q_sim_batch.calc_dynamics_parallel(
#             q_batch, q_a_batch, sim_params
#         )
# Option 2
q_next_batch, A_batch, B_batch, is_valid_batch = q_sim_batch.calc_dynamics_parallel(
            q_batch, u_batch, sim_params
        )
print("q_next_batch\n", q_next_batch)
A_batch = np.array(A_batch)
A_qunext_qa_batch = A_batch[:,q_sim.get_q_u_indices_into_q(),:][:,:,q_sim.get_q_a_indices_into_q()]
A_mean = np.mean(A_qunext_qa_batch, axis=0)
print("\nA_qunext_qa_mean\n", A_mean)

qu_next_batch = q_next_batch[:,q_sim.get_q_u_indices_into_q()]
qu_next_bar = np.mean(qu_next_batch, axis=0)
A_zeroth_order = np.mean(np.einsum('bi,bj->bij', qu_next_batch - qu_next_bar, w), axis=0) * (1/w_std**2)
print("\nA_zeroth_order\n", A_zeroth_order)

# Finite difference
h = 1e-7
v = np.array([1,0])
q_fdp = np.concatenate((q_u, q_a + h*v)) 
q_next_1 = q_sim.calc_dynamics(q_fdp, u, sim_params)
q_u_next_1 = q_next_1[q_sim.get_q_u_indices_into_q()]
q_fdn = np.concatenate((q_u, q_a - h*v)) 
q_next_2 = q_sim.calc_dynamics(q_fdn, u, sim_params)
q_u_next_2 = q_next_2[q_sim.get_q_u_indices_into_q()]
dx_dq1 = (q_u_next_1 - q_u_next_2) / (2*h)
print("\ndx_dq1\n", dx_dq1[:, np.newaxis])

q_next_batch
 [[ 1.50881311e-02  1.06123159e-03  1.65008348e-03 -1.00150881e+00
  -1.06123138e-04]
 [ 1.52462009e-02  1.15409430e-03  1.83838247e-03 -1.00152462e+00
  -1.15409407e-04]
 [ 1.51217732e-02  4.79102123e-03  6.79094354e-03 -1.00151218e+00
  -4.79102028e-04]
 ...
 [ 1.57929176e-02 -4.54436037e-04 -7.88505878e-04 -1.00157929e+00
   4.54435946e-05]
 [ 1.79826435e-02 -4.29603134e-03 -9.02792883e-03 -1.00179826e+00
   4.29603048e-04]
 [ 1.48015736e-02  1.68052335e-03  2.47263055e-03 -1.00148016e+00
  -1.68052302e-04]]

A_qunext_qa_mean
 [[-1.85022847e-02 -1.35425236e-04]
 [ 5.25223181e-05 -2.91995344e-02]
 [-5.27797693e-04 -4.31293571e-02]]

A_zeroth_order
 [[-0.01865099  0.00012324]
 [ 0.00045948 -0.03018136]
 [ 0.00012441 -0.04452897]]

dx_dq1
 [[-1.73089626e-02]
 [ 4.07181168e-07]
 [-1.94732558e-05]]


In [5]:
q_u = np.array([0, 0, 0])
q_a = np.array([-1, 0])
q = np.concatenate((q_u, q_a)) # [box_x, box_y, box_theta, sphere_x, sphere_y]
u = q_a
q_sim_py.update_mbp_positions_from_vector(q)
q_sim_py.draw_current_configuration()

q_next = q_sim.calc_dynamics(q, u, sim_params)
A = q_sim.get_Dq_nextDq()
B = q_sim.get_Dq_nextDqa_cmd()
print("q_next:\n",q_next)
print("A:\n",A)
print("B:\n",B)

A_qunext_qa = A[q_sim.get_q_u_indices_into_q(),:][:,q_sim.get_q_a_indices_into_q()]
B_qunext_u = B[q_sim.get_q_u_indices_into_q(),:]
print("A_qunext_qa:\n",A_qunext_qa)
print("B_qunext_u:\n",B_qunext_u)
delta = np.array([0.1, 0])
delta_qa = delta[:, np.newaxis]
delta_u = delta_qa
delta_qu_next = A_qunext_qa @ delta_qa + B_qunext_u @ delta_u
print("delta_qu_next:\n",delta_qu_next)

delta_q = np.concatenate((q_u, q_a + delta))
q_next_2 = q_sim.calc_dynamics(delta_q, u + delta, sim_params)
print("q_next_2:\n",q_next_2)

q_next:
 [ 1.51913121e-02  9.52584963e-14 -4.55596098e-12 -1.00151913e+00
 -9.52588625e-15]
A:
 [[ 9.90553945e-01  7.21245091e-14 -3.89043892e-14 -1.73091159e-02
  -1.61535033e-14]
 [ 8.05151786e-14  1.00750154e+00  7.68977989e-03 -2.45426840e-14
  -2.75574873e-02]
 [-1.87225505e-12  2.18569667e-01  7.81430334e-01 -6.54948398e-13
  -4.37494156e-02]
 [ 9.44605539e-04 -7.21243685e-15  3.89042261e-15  1.73091159e-03
   1.61533403e-15]
 [-8.05151655e-15 -7.50153696e-04 -7.68977835e-04  2.45441372e-15
   2.75574818e-03]]
B:
 [[ 2.67551713e-02 -5.59710058e-14]
 [-5.59724946e-14  2.00559489e-02]
 [ 2.52720345e-12 -1.74820251e-01]
 [ 9.97324483e-01  5.59710282e-15]
 [ 5.59710282e-15  9.97994406e-01]]
A_qunext_qa:
 [[-1.73091159e-02 -1.61535033e-14]
 [-2.45426840e-14 -2.75574873e-02]
 [-6.54948398e-13 -4.37494156e-02]]
B_qunext_u:
 [[ 2.67551713e-02 -5.59710058e-14]
 [-5.59724946e-14  2.00559489e-02]
 [ 2.52720345e-12 -1.74820251e-01]]
delta_qu_next:
 [[ 9.44605539e-04]
 [-8.05151786e-15]
 [ 1.

# Explanation of the issue
Consider the initial configuration where the box is at the origin and the sphere is at x=-1 (y=0).

In [6]:
q_u = np.array([0, 0, 0])
q_a = np.array([-1, 0])

print("box position: ", q_u)
print("sphere position: ", q_a)

box position:  [0 0 0]
sphere position:  [-1  0]


If we evaluate the dynamics at in this configuration (with a nominal position command of u = sphere position) with smoothed dynamics we see that the sphere exerts a force from a distance on the box pushing it to the right by 0.015, and the sphere correspondingly experiences a force pushing it left by 0.002.

In [7]:

q = np.concatenate((q_u, q_a)) # [box_x, box_y, box_theta, sphere_x, sphere_y]
u = q_a
q_sim_py.update_mbp_positions_from_vector(q)
q_sim_py.draw_current_configuration()
q_next = q_sim.calc_dynamics(q, u, sim_params)
print("q_next:\n",q_next)

q_next:
 [ 1.51913121e-02  9.52584963e-14 -4.55596098e-12 -1.00151913e+00
 -9.52588625e-15]


The corresponding A and B matrices tell us how the full state would change wrt changes in the configuration, at this current configuration (it's the linearization at this point)

In [8]:
A = q_sim.get_Dq_nextDq()
B = q_sim.get_Dq_nextDqa_cmd()
print("A:\n",A)
print("B:\n",B)

A:
 [[ 9.90553945e-01  7.21245091e-14 -3.89043892e-14 -1.73091159e-02
  -1.61535033e-14]
 [ 8.05151786e-14  1.00750154e+00  7.68977989e-03 -2.45426840e-14
  -2.75574873e-02]
 [-1.87225505e-12  2.18569667e-01  7.81430334e-01 -6.54948398e-13
  -4.37494156e-02]
 [ 9.44605539e-04 -7.21243685e-15  3.89042261e-15  1.73091159e-03
   1.61533403e-15]
 [-8.05151655e-15 -7.50153696e-04 -7.68977835e-04  2.45441372e-15
   2.75574818e-03]]
B:
 [[ 2.67551713e-02 -5.59710058e-14]
 [-5.59724946e-14  2.00559489e-02]
 [ 2.52720345e-12 -1.74820251e-01]
 [ 9.97324483e-01  5.59710282e-15]
 [ 5.59710282e-15  9.97994406e-01]]


We are interested in how the state of the box changes for a slight change in the nominal position of the sphere (and nominal position command - such that the sphere is commanded to not move basically), and so we can extract out the relevant parts of the A and B matrices that correspond to these mappings.

In [9]:
A_qunext_qa = A[q_sim.get_q_u_indices_into_q(),:][:,q_sim.get_q_a_indices_into_q()]
B_qunext_u = B[q_sim.get_q_u_indices_into_q(),:]
print("A_qunext_qa:\n",A_qunext_qa)
print("B_qunext_u:\n",B_qunext_u)

A_qunext_qa:
 [[-1.73091159e-02 -1.61535033e-14]
 [-2.45426840e-14 -2.75574873e-02]
 [-6.54948398e-13 -4.37494156e-02]]
B_qunext_u:
 [[ 2.67551713e-02 -5.59710058e-14]
 [-5.59724946e-14  2.00559489e-02]
 [ 2.52720345e-12 -1.74820251e-01]]


To verify that these sub matrices make sense, let's consider a small delta [0.1, 0] in the position of the sphere (and corresponding delta in absolute position command). So this would mean if if we shifted the nominal positions and position commands to the right by 0.1, the sub A and B matrices should tell us the additional amount that the box would move. Thinking through it, the box should move slightly more to the right (positive x). Let's verify that this is what our sub A and B matrices tell us

In [10]:
delta = np.array([0.1, 0])
delta_qa = delta[:, np.newaxis]
delta_u = delta_qa
delta_qu_next = A_qunext_qa @ delta_qa + B_qunext_u @ delta_u
print("delta_qu_next:\n",delta_qu_next)

delta_qu_next:
 [[ 9.44605539e-04]
 [-8.05151786e-15]
 [ 1.87225505e-13]]


Perfect! It shows us that we expect the box to move 0.001 more to the left, which appears to be directionally correct. Now feeding our nominal + delta configuration to evaluate the dynamics, let's see that we get that this is 0.001 different from the nominal configuration:

In [11]:
delta_q = np.concatenate((q_u, q_a + delta))
q_next_2 = q_sim.calc_dynamics(delta_q, u + delta, sim_params)
print("q_next: ", q_next)
print("q_next_2: ",q_next_2)

q_next:  [ 1.51913121e-02  9.52584963e-14 -4.55596098e-12 -1.00151913e+00
 -9.52588625e-15]
q_next_2:  [ 1.63317451e-02 -8.97522160e-14  2.91356364e-12 -9.01633175e-01
  8.97524941e-15]


Our results are exactly what we expect.